In [1]:
# Set up autoreloading of modules so that I can debug code in external files
%load_ext autoreload
%autoreload 2

In [57]:
import pandas as pd
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt

import bydatetime
import hillpylib as hm
from pandas import Timestamp

# Let's check what version of pandas, numpy and matplotlib we are using
print ("pandas version ", pd.__version__)
print ("numpy version ", np.version.version)
print ("matplotlib version ", mp.__version__)

pandas version  0.15.2
numpy version  1.9.2
matplotlib version  1.4.3


# Put it all together

Below I've strung together all the pieces to do an entire Hillmaker run. Change inputs as needed (e.g. scenario_name and associated parameter values) and run all the cells below. You can skip rereading the main input file if that isn't changing.

## Read main stop data file

In [3]:
file_stopdata = 'data/ShortStay.csv'
df = pd.read_csv(file_stopdata, parse_dates=['InRoomTS','OutRoomTS'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59877 entries, 0 to 59876
Data columns (total 4 columns):
PatID        59877 non-null int64
InRoomTS     59877 non-null datetime64[ns]
OutRoomTS    59877 non-null datetime64[ns]
PatType      59877 non-null object
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 2.3+ MB


## Set input parameters

In [99]:
# Required inputs
scenario_name = 'sstest_60'
in_fld_name = 'InRoomTS'
out_fld_name = 'OutRoomTS'
cat_fld_name = 'PatType'
start_analysis = '1/1/1996'
end_analysis = '3/30/1996 23:45'


# Optional inputs

# This next field wasn't in original Hillmaker. Use it to specify the name to use for the overall totals.
# At this point the totals actually aren't being calculated.
tot_fld_name = 'SSU'

bin_size_mins = 60

includecats = ['ART','IVT']

## Convert string dates to actual datetimes
start_analysis_dt = pd.Timestamp(start_analysis)
end_analysis_dt = pd.Timestamp(end_analysis)

# Mapper from weekday integer to string
daynum_to_dayname = {0: 'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri', 5: 'Sat', 6: 'Sun'}


## Create the by datetime table

In [13]:
df2 = df[df['PatType'].isin(includecats)]

In [14]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38940 entries, 0 to 59875
Data columns (total 4 columns):
PatID        38940 non-null int64
InRoomTS     38940 non-null datetime64[ns]
OutRoomTS    38940 non-null datetime64[ns]
PatType      38940 non-null object
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 1.5+ MB


In [15]:
df2.groupby('PatType').describe()

PatID
PatType                    
ART     count   5761.000000
        mean   29665.735983
        std    17236.415692
        min       10.000000
        25%    14738.000000
        50%    29511.000000
        75%    44784.000000
        max    59826.000000
IVT     count  33179.000000
        mean   30015.498357
        std    17270.541655
        min        1.000000
        25%    15099.500000
        50%    29956.000000
        75%    45027.500000
        max    59876.000000

In [16]:
df = df[df['PatType'].isin(includecats)]

In [17]:
df.groupby('PatType').describe()

PatID
PatType                    
ART     count   5761.000000
        mean   29665.735983
        std    17236.415692
        min       10.000000
        25%    14738.000000
        50%    29511.000000
        75%    44784.000000
        max    59826.000000
IVT     count  33179.000000
        mean   30015.498357
        std    17270.541655
        min        1.000000
        25%    15099.500000
        50%    29956.000000
        75%    45027.500000
        max    59876.000000

In [102]:
bydt_df = bydatetime.make_bydatetime(df,
                                     in_fld_name,
                                     out_fld_name,
                                     cat_fld_name,
                                     start_analysis,
                                     end_analysis,
                                     tot_fld_name,
                                     bin_size_mins,
                                     categories=includecats)

rng_bydt created: 0.0020
using specified categories: 0.0130
Seeded bydatetime DataFrame created: 0.0353
dayofweek, bin_of_day, bin_of_week computed: 0.1410
Multi-index on bydatetime DataFrame created: 0.1438
Multi-index fully lexsorted: 0.1460
Latest edits at 2015-06-25 14:42:58.082470
Num inner: 12852
Done processing 12852 stop recs: 9.5413
Done adding totals: 9.6173


In [6]:
bydt_df.dtypes

category               object
datetime       datetime64[ns]
arrivals              float64
departures            float64
occupancy             float64
day_of_week             int64
bin_of_day              int64
bin_of_week             int64
dtype: object

In [50]:
bydt_df

category            datetime  arrivals  \
category datetime                                                     
ART      1996-01-01 00:00:00      ART 1996-01-01 00:00:00         0   
         1996-01-01 01:00:00      ART 1996-01-01 01:00:00         0   
         1996-01-01 02:00:00      ART 1996-01-01 02:00:00         0   
         1996-01-01 03:00:00      ART 1996-01-01 03:00:00         0   
         1996-01-01 04:00:00      ART 1996-01-01 04:00:00         0   
         1996-01-01 05:00:00      ART 1996-01-01 05:00:00         0   
         1996-01-01 06:00:00      ART 1996-01-01 06:00:00         0   
         1996-01-01 07:00:00      ART 1996-01-01 07:00:00         0   
         1996-01-01 08:00:00      ART 1996-01-01 08:00:00         0   
         1996-01-01 09:00:00      ART 1996-01-01 09:00:00         0   
         1996-01-01 10:00:00      ART 1996-01-01 10:00:00         0   
         1996-01-01 11:00:00      ART 1996-01-01 11:00:00         0   
         1996-01-01 12:00:00      ART 1996-01-01 12:00:00         0   
         1996-01-01 13:00:00      ART 1996-01-01 13:00:00         0   
         1996-01-01 14:00:00      ART 1996-01-01 14:00:00         0   
         1996-01-01 15:00:00      ART 1996-01-01 15:00:00         0   
         1996-01-01 16:00:00      ART 1996-01-01 16:00:00         0   
         1996-01-01 17:00:00      ART 1996-01-01 17:00:00         0   
         1996-01-01 18:00:00      ART 1996-01-01 18:00:00         0   
         1996-01-01 19:00:00      ART 1996-01-01 19:00:00         0   
         1996-01-01 20:00:00      ART 1996-01-01 20:00:00         0   
         1996-01-01 21:00:00      ART 1996-01-01 21:00:00         0   
         1996-01-01 22:00:00      ART 1996-01-01 22:00:00         0   
         1996-01-01 23:00:00      ART 1996-01-01 23:00:00         0   
         1996-01-02 00:00:00      ART 1996-01-02 00:00:00         0   
         1996-01-02 01:00:00      ART 1996-01-02 01:00:00         0   
         1996-01-02 02:00:00      ART 1996-01-02 02:00:00         0   
         1996-01-02 03:00:00      ART 1996-01-02 03:00:00         0   
         1996-01-02 04:00:00      ART 1996-01-02 04:00:00         0   
         1996-01-02 05:00:00      ART 1996-01-02 05:00:00         0   
...                               ...                 ...       ...   
IVT      1996-03-29 18:00:00      IVT 1996-03-29 18:00:00         9   
         1996-03-29 19:00:00      IVT 1996-03-29 19:00:00         2   
         1996-03-29 20:00:00      IVT 1996-03-29 20:00:00         1   
         1996-03-29 21:00:00      IVT 1996-03-29 21:00:00         0   
         1996-03-29 22:00:00      IVT 1996-03-29 22:00:00         0   
         1996-03-29 23:00:00      IVT 1996-03-29 23:00:00         0   
         1996-03-30 00:00:00      IVT 1996-03-30 00:00:00         0   
         1996-03-30 01:00:00      IVT 1996-03-30 01:00:00         0   
         1996-03-30 02:00:00      IVT 1996-03-30 02:00:00         0   
         1996-03-30 03:00:00      IVT 1996-03-30 03:00:00         0   
         1996-03-30 04:00:00      IVT 1996-03-30 04:00:00         0   
         1996-03-30 05:00:00      IVT 1996-03-30 05:00:00         0   
         1996-03-30 06:00:00      IVT 1996-03-30 06:00:00         2   
         1996-03-30 07:00:00      IVT 1996-03-30 07:00:00         3   
         1996-03-30 08:00:00      IVT 1996-03-30 08:00:00         3   
         1996-03-30 09:00:00      IVT 1996-03-30 09:00:00         3   
         1996-03-30 10:00:00      IVT 1996-03-30 10:00:00         5   
         1996-03-30 11:00:00      IVT 1996-03-30 11:00:00         2   
         1996-03-30 12:00:00      IVT 1996-03-30 12:00:00         3   
         1996-03-30 13:00:00      IVT 1996-03-30 13:00:00         0   
         1996-03-30 14:00:00      IVT 1996-03-30 14:00:00         0   
         1996-03-30 15:00:00      IVT 1996-03-30 15:00:00         0   
         1996-03-30 16:00:00      IVT 1996-03-30 16:00:00         0   
         1996-03-30 17:00:00      IVT 1996-03-30 17:0

In [51]:
bydt_group = bydt_df.groupby(['datetime'])

In [54]:
tot_arrivals = bydt_group.arrivals.sum()
tot_departures = bydt_group.departures.sum()
tot_occ = bydt_group.occupancy.sum()

#bydt_totals = pd.DataFrame(tot_arrivals)


In [84]:
tot_data = [tot_arrivals,tot_departures,tot_occ]
tot_df = pd.concat(tot_data, axis = 1, keys = [s.name for s in tot_data])

In [90]:
tot_data = [tot_arrivals,tot_departures,tot_occ]
tot_df = pd.concat(tot_data, axis = 1, keys = [s.name for s in tot_data])
tot_df['day_of_week'] = tot_df.index.map(lambda x: x.weekday())
tot_df['bin_of_day'] =  tot_df.index.map(lambda x: hm.bin_of_day(x,bin_size_mins))
tot_df['bin_of_week'] = tot_df.index.map(lambda x: hm.bin_of_week(x,bin_size_mins))

In [91]:
tot_df['category'] = tot_fld_name
tot_df.set_index('category', append=True, inplace=True, drop=False)
tot_df = tot_df.reorder_levels(['category', 'datetime'])
tot_df['datetime'] = tot_df.index.levels[1]

In [ ]:
tot_df

In [94]:
tot_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2160 entries, (Total, 1996-01-01 00:00:00) to (Total, 1996-03-30 23:00:00)
Data columns (total 8 columns):
arrivals       2160 non-null float64
departures     2160 non-null float64
occupancy      2160 non-null float64
day_of_week    2160 non-null int64
bin_of_day     2160 non-null int64
bin_of_week    2160 non-null int64
category       2160 non-null object
datetime       2160 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(3), object(1)
memory usage: 158.4+ KB


In [96]:
bydt_df = pd.concat([bydt_df,tot_df])

In [98]:
bydt_df.tail(n=25)

arrivals  bin_of_day  bin_of_week category  \
category datetime                                                          
Total    1996-03-29 23:00:00         0          23          119    Total   
         1996-03-30 00:00:00         0           0          120    Total   
         1996-03-30 01:00:00         0           1          121    Total   
         1996-03-30 02:00:00         0           2          122    Total   
         1996-03-30 03:00:00         0           3          123    Total   
         1996-03-30 04:00:00         0           4          124    Total   
         1996-03-30 05:00:00         0           5          125    Total   
         1996-03-30 06:00:00         2           6          126    Total   
         1996-03-30 07:00:00         3           7          127    Total   
         1996-03-30 08:00:00         3           8          128    Total   
         1996-03-30 09:00:00         3           9          129    Total   
         1996-03-30 10:00:00         5          10          130    Total   
         1996-03-30 11:00:00         2          11          131    Total   
         1996-03-30 12:00:00         3          12          132    Total   
         1996-03-30 13:00:00         0          13          133    Total   
         1996-03-30 14:00:00         0          14          134    Total   
         1996-03-30 15:00:00         0          15          135    Total   
         1996-03-30 16:00:00         0          16          136    Total   
         1996-03-30 17:00:00         0          17          137    Total   
         1996-03-30 18:00:00         0          18          138    Total   
         1996-03-30 19:00:00         0          19          139    Total   
         1996-03-30 20:00:00         0          20          140    Total   
         1996-03-30 21:00:00         0          21          141    Total   
         1996-03-30 22:00:00         0          22          142    Total   
         1996-03-30 23:00:00         0          23          143    Total   

                                        datetime  day_of_week  departures  \
category datetime                                                           
Total    1996-03-29 23:00:00 1996-03-29 23:00:00            4           0   
         1996-03-30 00:00:00 1996-03-30 00:00:00            5           0   
         1996-03-30 01:00:00 1996-03-30 01:00:00            5           0   
         1996-03-30 02:00:00 1996-03-30 02:00:00            5           0   
         1996-03-30 03:00:00 1996-03-30 03:00:00            5           0   
         1996-03-30 04:00:00 1996-03-30 04:00:00            5           0   
         1996-03-30 05:00:00 1996-03-30 05:00:00            5           0   
         1996-03-30 06:00:00 1996-03-30 06:00:00            5           0   
         1996-03-30 07:00:00 1996-03-30 07:00:00            5           1   
         1996-03-30 08:00:00 1996-03-30 08:00:00            5           4   
         1996-03-30 09:00:00 1996-03-30 09:00:00            5           4   
         1996-03-30 10:00:00 1996-03-30 10:00:00            5           2   
         1996-03-30 11:00:00 1996-03-30 11:00:00            5           6   
         1996-03-30 12:00:00 1996-03-30 12:00:00            5           1   
         1996-03-30 13:00:00 1996-03-30 13:00:00            5           3   
         1996-03-30 14:00:00 1996-03-30 14:00:00            5           0   
         1996-03-30 15:00:00 1996-03-30 15:00:00            5           0   
         1996-03-30 16:00:00 1996-03-30 16:00:00            5           0   
         1996-03-30 17:00:00 1996-03-30 17:00:00            5           0   
         1996-03-30 18:00:00 1996-03-30 18:00:00            5           0   
         1996-03-30 19:00:00 1996-03-30 19:00:00            5           0   
         1996-03-30 20:00:00 1996-03-30 20:00:00            5           0   
         1996-03-30 21:00:00 1996-03-30 21:00:00            5           0   
         1996-03-30 22:00:00 1996-03-30 22:00:00    

In [7]:
def bydt_totals(group):
    return {'category': totals, stub+'mean': group.mean(), 
            stub+'min': group.min(),
            stub+'max': group.max(), 'stdev': group.std(), 
            stub+'p50': group.quantile(0.5), stub+'p55': group.quantile(0.55),
            stub+'p60': group.quantile(0.6), stub+'p65': group.quantile(0.65),
            stub+'p70': group.quantile(0.7), stub+'p75': group.quantile(0.75),
            stub+'p80': group.quantile(0.8), stub+'p85': group.quantile(0.85),
            stub+'p90': group.quantile(0.9), stub+'p95': group.quantile(0.95),
            stub+'p975': group.quantile(0.975), 
            stub+'p99': group.quantile(0.99)}

## Compute summary stats

In [23]:
def get_occstats(group, stub=''):
    return {stub+'count': group.count(), stub+'mean': group.mean(), 
            stub+'min': group.min(),
            stub+'max': group.max(), 'stdev': group.std(), 
            stub+'p50': group.quantile(0.5), stub+'p55': group.quantile(0.55),
            stub+'p60': group.quantile(0.6), stub+'p65': group.quantile(0.65),
            stub+'p70': group.quantile(0.7), stub+'p75': group.quantile(0.75),
            stub+'p80': group.quantile(0.8), stub+'p85': group.quantile(0.85),
            stub+'p90': group.quantile(0.9), stub+'p95': group.quantile(0.95),
            stub+'p975': group.quantile(0.975), 
            stub+'p99': group.quantile(0.99)}

In [24]:
bydt_dfgrp2 = bydt_df.groupby(['category','day_of_week','bin_of_day'])

occ_stats = bydt_dfgrp2['occupancy'].apply(get_occstats)
arr_stats = bydt_dfgrp2['arrivals'].apply(get_occstats)
dep_stats = bydt_dfgrp2['departures'].apply(get_occstats)

occ_stats_summary = occ_stats.unstack()
arr_stats_summary = arr_stats.unstack()
dep_stats_summary = dep_stats.unstack()



In [32]:
occ_stats.dtype

dtype('float64')

In [33]:
type(occ_stats)

pandas.core.series.Series

In [36]:
occ_stats_summary.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 336 entries, (ART, 0, 0) to (IVT, 6, 23)
Data columns (total 17 columns):
count    336 non-null float64
max      336 non-null float64
mean     336 non-null float64
min      336 non-null float64
p50      336 non-null float64
p55      336 non-null float64
p60      336 non-null float64
p65      336 non-null float64
p70      336 non-null float64
p75      336 non-null float64
p80      336 non-null float64
p85      336 non-null float64
p90      336 non-null float64
p95      336 non-null float64
p975     336 non-null float64
p99      336 non-null float64
stdev    336 non-null float64
dtypes: float64(17)
memory usage: 46.2+ KB


## Write summaries and by datetime out to CSV

In [9]:
file_bydt_csv = 'testing/bydate_' + scenario_name + '.csv'
bydt_df.to_csv(file_bydt_csv, index=False)

file_occ_csv = 'testing/occ_stats_' + scenario_name + '.csv'
file_arr_csv = 'testing/arr_stats_' + scenario_name + '.csv'
file_dep_csv = 'testing/dep_stats_' + scenario_name + '.csv'

occ_stats_summary.to_csv(file_occ_csv)
arr_stats_summary.to_csv(file_arr_csv)
dep_stats_summary.to_csv(file_dep_csv)

## Debugging

In [10]:
ts = pd.Timestamp('19960103 00:00:00')
print(ts)

1996-01-03 00:00:00


In [11]:
df_ART = df[(df.PatType == 'ART') & (df.InRoomTS < ts)]

In [12]:
df_ART.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 9 to 174
Data columns (total 4 columns):
PatID        30 non-null int64
InRoomTS     30 non-null datetime64[ns]
OutRoomTS    30 non-null datetime64[ns]
PatType      30 non-null object
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 1.2+ KB


In [13]:
df_ART

,PatID,InRoomTS,OutRoomTS,PatType
9,10,1996-01-02 06:35:00,1996-01-02 08:30:00,ART
12,13,1996-01-02 06:55:00,1996-01-02 08:45:00,ART
15,16,1996-01-02 07:18:00,1996-01-02 08:53:00,ART
17,18,1996-01-02 07:23:00,1996-01-02 08:30:00,ART
21,22,1996-01-02 07:33:00,1996-01-02 08:45:00,ART
24,25,1996-01-02 07:40:00,1996-01-02 08:55:00,ART
25,26,1996-01-02 07:50:00,1996-01-02 10:10:00,ART
27,28,1996-01-02 07:52:00,1996-01-02 09:10:00,ART
30,31,1996-01-02 08:00:00,1996-01-02 09:00:00,ART
31,32,1996-01-02 08:02:00,1996-01-02 10:00:00,ART


In [14]:
bydt_df.head()

category            datetime  arrivals  \
category datetime                                                     
ART      1996-01-01 00:00:00      ART 1996-01-01 00:00:00         0   
         1996-01-01 01:00:00      ART 1996-01-01 01:00:00         0   
         1996-01-01 02:00:00      ART 1996-01-01 02:00:00         0   
         1996-01-01 03:00:00      ART 1996-01-01 03:00:00         0   
         1996-01-01 04:00:00      ART 1996-01-01 04:00:00         0   

                              departures  occupancy  day_of_week  bin_of_day  \
category datetime                                                              
ART      1996-01-01 00:00:00           0          0            0           0   
         1996-01-01 01:00:00           0          0            0           1   
         1996-01-01 02:00:00           0          0            0           2   
         1996-01-01 03:00:00           0          0            0           3   
         1996-01-01 04:00:00           0          0            0           4   

                              bin_of_week  
category datetime                          
ART      1996-01-01 00:00:00            0  
         1996-01-01 01:00:00            1  
         1996-01-01 02:00:00            2  
         1996-01-01 03:00:00            3  
         1996-01-01 04:00:00            4

In [15]:
bydt_df[25:50]

category            datetime  arrivals  \
category datetime                                                     
ART      1996-01-02 01:00:00      ART 1996-01-02 01:00:00         0   
         1996-01-02 02:00:00      ART 1996-01-02 02:00:00         0   
         1996-01-02 03:00:00      ART 1996-01-02 03:00:00         0   
         1996-01-02 04:00:00      ART 1996-01-02 04:00:00         0   
         1996-01-02 05:00:00      ART 1996-01-02 05:00:00         0   
         1996-01-02 06:00:00      ART 1996-01-02 06:00:00         2   
         1996-01-02 07:00:00      ART 1996-01-02 07:00:00         6   
         1996-01-02 08:00:00      ART 1996-01-02 08:00:00         5   
         1996-01-02 09:00:00      ART 1996-01-02 09:00:00         5   
         1996-01-02 10:00:00      ART 1996-01-02 10:00:00         2   
         1996-01-02 11:00:00      ART 1996-01-02 11:00:00         5   
         1996-01-02 12:00:00      ART 1996-01-02 12:00:00         1   
         1996-01-02 13:00:00      ART 1996-01-02 13:00:00         3   
         1996-01-02 14:00:00      ART 1996-01-02 14:00:00         0   
         1996-01-02 15:00:00      ART 1996-01-02 15:00:00         1   
         1996-01-02 16:00:00      ART 1996-01-02 16:00:00         0   
         1996-01-02 17:00:00      ART 1996-01-02 17:00:00         0   
         1996-01-02 18:00:00      ART 1996-01-02 18:00:00         0   
         1996-01-02 19:00:00      ART 1996-01-02 19:00:00         0   
         1996-01-02 20:00:00      ART 1996-01-02 20:00:00         0   
         1996-01-02 21:00:00      ART 1996-01-02 21:00:00         0   
         1996-01-02 22:00:00      ART 1996-01-02 22:00:00         0   
         1996-01-02 23:00:00      ART 1996-01-02 23:00:00         0   
         1996-01-03 00:00:00      ART 1996-01-03 00:00:00         0   
         1996-01-03 01:00:00      ART 1996-01-03 01:00:00         0   

                              departures  occupancy  day_of_week  bin_of_day  \
category datetime                                                              
ART      1996-01-02 01:00:00           0   0.000000            1           1   
         1996-01-02 02:00:00           0   0.000000            1           2   
         1996-01-02 03:00:00           0   0.000000            1           3   
         1996-01-02 04:00:00           0   0.000000            1           4   
         1996-01-02 05:00:00           0   0.000000            1           5   
         1996-01-02 06:00:00           0   0.500000            1           6   
         1996-01-02 07:00:00           0   4.400000            1           7   
         1996-01-02 08:00:00           6   9.716667            1           8   
         1996-01-02 09:00:00           4   6.933333            1           9   
         1996-01-02 10:00:00           5   4.833333            1          10   
         1996-01-02 11:00:00           2   5.516667            1          11   
         1996-01-02 12:00:00           6   5.016667            1          12   
         1996-01-02 13:00:00           3   2.900000            1          13   
         1996-01-02 14:00:00           1   2.516667            1          14   
         1996-01-02 15:00:00           2   1.483333            1          15   
         1996-01-02 16:00:00           0   1.000000            1          16   
         1996-01-02 17:00:00           1   0.000000            1          17   
         1996-01-02 18:00:00           0   0.000000            1          18   
         1996-01-02 19:00:00           0   0.000000            1          19   
         1996-01-02 20:00:00           0   0.000000            1          20   
         1996-01-02 21:00:00           0   0.000000            1          21   
         1996-01-02 22:00:00           0   0.000000            1          22   
         1996-01-02 23:00:00           0   0.000000            1          23   
         1996-01-03 00:00:00           0   0.000000            2           0   
         1996-01-03 01:00:00   